In [1]:
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH17-LangGraph-Structures")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH17-LangGraph-Structures


# 기본 PDF 기반 Retrieval Chain 생성

In [2]:
from rag.pdf import PDFRetrievalChain

# PDF 문서를 로드합니다.
pdf = PDFRetrievalChain(["data/SPRI_AI_Brief_2023년12월호_F.pdf"]).create_chain()

# retriever와 chain을 생성합니다.
pdf_retriever = pdf.retriever
pdf_chain = pdf.chain

# State 정의

In [3]:
from typing import Annotated, TypedDict
from langgraph.graph.message import add_messages


# GraphState 상태 정의
class GraphState(TypedDict):
    question: Annotated[str, "Question"]  # 질문
    context: Annotated[str, "Context"]  # 문서의 검색 결과
    answer: Annotated[str, "Answer"]  # 답변
    messages: Annotated[list, add_messages]  # 메시지(누적되는 list)
    relevance: Annotated[str, "Relevance"]  # 관련성

# 노드(Node) 정의

In [4]:
from langchain_openai import ChatOpenAI
from langchain_teddynote.evaluator import GroundednessChecker
from langchain_teddynote.messages import messages_to_history
from rag.utils import format_docs


# 문서 검색 노드
def retrieve_document(state: GraphState) -> GraphState:
    # 질문을 상태에서 가져옵니다.
    latest_question = state["question"]

    # 문서에서 검색하여 관련성 있는 문서를 찾습니다.
    retrieved_docs = pdf_retriever.invoke(latest_question)

    # 검색된 문서를 형식화합니다.(프롬프트 입력으로 넣어주기 위함)
    retrieved_docs = format_docs(retrieved_docs)

    # 검색된 문서를 context 키에 저장합니다.
    return GraphState(context=retrieved_docs)


# 답변 생성 노드
def llm_answer(state: GraphState) -> GraphState:
    # 질문을 상태에서 가져옵니다.
    latest_question = state["question"]

    # 검색된 문서를 상태에서 가져옵니다.
    context = state["context"]

    # 체인을 호출하여 답변을 생성합니다.
    response = pdf_chain.invoke(
        {
            "question": latest_question,
            "context": context,
            "chat_history": messages_to_history(state["messages"]),
        }
    )
    # 생성된 답변, (유저의 질문, 답변) 메시지를 상태에 저장합니다.
    return GraphState(
        answer=response, messages=[("user", latest_question), ("assistant", response)]
    )


# 관련성 체크 노드
def relevance_check(state: GraphState) -> GraphState:
    # 관련성 평가기를 생성합니다.
    question_answer_relevant = GroundednessChecker(
        llm=ChatOpenAI(model="gpt-4o-mini", temperature=0), target="question-retrieval"
    ).create()

    # 관련성 체크를 실행("yes" or "no")
    response = question_answer_relevant.invoke(
        {"question": state["question"], "context": state["context"]}
    )

    print("==== [RELEVANCE CHECK] ====")
    print(response.score)

    # 참고: 여기서의 관련성 평가기는 각자의 Prompt 를 사용하여 수정할 수 있습니다. 여러분들의 Groundedness Check 를 만들어 사용해 보세요!
    return GraphState(relevance=response.score)


# 관련성 체크하는 함수(router)
def is_relevant(state: GraphState) -> GraphState:
    return state["relevance"]

# 검색 노드 추가

In [5]:
from langchain_teddynote.tools.tavily import TavilySearch

# 검색 도구 생성
tavily_tool = TavilySearch()

search_query = "실시간 미국 대선 투표 결과는?"

# 다양한 파라미터를 사용한 검색 예제
search_result = tavily_tool.search(
    query=search_query,  # 검색 쿼리
    topic="news",  # 일반 주제
    days=1,  # 최근 1일 내 검색
    max_results=3,  # 최대 검색 결과
    format_output=True,  # 결과 포맷팅
)

print("\n".join(search_result))

<document><title>US stocks rally as Trump signals thaw in trade war, Tesla shares soar - ABC News</title><url>https://abcnews.go.com/Business/us-stocks-rally-trump-signals-thaw-trade-war/story?id=121078759</url><content>US stocks rally as Trump signals thaw in trade war, Tesla shares soar - ABC News US stocks rally as Trump signals thaw in trade war, Tesla shares soar U.S. stocks rallied on Wednesday, one day after President Donald Trump said tariffs on China would \"come down substantially.\" Amazon's Worst Nightmare: Thousands Canceling Prime for This Clever HackOnline Shopping Tools / [Sponsored](https://popup.taboola.com/en/?template=colorbox&utm_source=abcnews-abcnews&utm_medium=referral&utm_content=thumbnails-a-02:Below%20Article%20Thumbnails%20|%20Card%202:)[Sponsored](https://popup.taboola.com/en/?template=colorbox&utm_source=abcnews-abcnews&utm_medium=referral&utm_content=thumbnails-a-02:Below%20Article%20Thumbnails%20|%20Card%202:) Social Security Recipients Under $2,384/Mo N

In [6]:
# Web Search 노드
def web_search(state: GraphState) -> GraphState:
    # 검색 도구 생성
    tavily_tool = TavilySearch()

    search_query = state["question"]

    # 다양한 파라미터를 사용한 검색 예제
    search_result = tavily_tool.search(
        query=search_query,  # 검색 쿼리
        topic="news",  # 일반 주제
        days=1,
        max_results=3,  # 최대 검색 결과
        format_output=True,  # 결과 포맷팅
    )

    return GraphState(context="\n".join(search_result))

In [8]:
from langgraph.graph import END, StateGraph
from langgraph.checkpoint.memory import MemorySaver

# 그래프 정의
workflow = StateGraph(GraphState)

# 노드 추가
workflow.add_node("retrieve", retrieve_document)
workflow.add_node("relevance_check", relevance_check)
workflow.add_node("llm_answer", llm_answer)

# Web Search 노드 추가
workflow.add_node("web_search", web_search)

# 엣지 추가
workflow.add_edge("retrieve", "relevance_check")  # 검색 -> 관련성 체크


# # 조건부 엣지를 추가합니다.
workflow.add_conditional_edges(
    "relevance_check",  # 관련성 체크 노드에서 나온 결과를 is_relevant 함수에 전달합니다.
    is_relevant,
    {
        "yes": "llm_answer",  # 관련성이 있으면 답변을 생성합니다.
        "no": "web_search",  # 관련성이 없으면 다시 검색합니다.
    },
)

workflow.add_edge("web_search", "llm_answer")  # 검색 -> 답변
workflow.add_edge("llm_answer", END)  # 답변 -> 종료

# 그래프 진입점 설정
workflow.set_entry_point("retrieve")

# 체크포인터 설정
memory = MemorySaver()

# 그래프 컴파일
app = workflow.compile(checkpointer=memory)

# 그래프 실행

In [9]:
from langchain_core.runnables import RunnableConfig
from langchain_teddynote.messages import stream_graph, random_uuid


# config 설정(재귀 최대 횟수, thread_id)
config = RunnableConfig(recursion_limit=10, configurable={"thread_id": random_uuid()})

# 질문 입력
inputs = GraphState(question="도널드 트럼프 대통령")

# 그래프 실행
stream_graph(app, inputs, config, ["relevance_check", "llm_answer"])


🔄 Node: relevance_check 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
{"score":"no==== [RELEVANCE CHECK] ====
no
"}
🔄 Node: llm_answer 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
도널드 트럼프 대통령은 현재 중국과의 무역 협상에서 "공정한 거래"를 목표로 하고 있으며, 중국 제품에 대한 높은 관세(현재 약 145%)를 "상당히 낮출 것"이라고 밝혔습니다. 그는 또한 우크라이나 전쟁과 관련하여 우크라이나 대통령 볼로디미르 젤렌스키를 비판하며, 그의 발언이 평화 협상에 해롭다고 언급했습니다. 트럼프의 지지율은 40%로 하락했으며, 이는 그의 두 번째 임기 100일을 앞두고 나타난 변화입니다.

**Source**
- [CNN - The latest on Trump’s presidency](https://www.cnn.com/politics/live-news/trump-presidency-news-04-23-25/index.html) 
- [CNN - Trump slams Zelensky for refusing to recognize Russian control of Crimea](https://edition.cnn.com/2025/04/23/europe/rubio-russia-ukraine-ceasefire-talks-intl-hnk/index.html) 
- [Politico - Trump to host dinner with top holders of his meme coin](https://www.politico.com/news/2025/04/23/trump-crypto-coin-dinner-00306707)

In [10]:
outputs = app.get_state(config).values

print(f'Question: {outputs["question"]}')
print("===" * 20)
print(f'Answer:\n{outputs["answer"]}')

Question: 도널드 트럼프 대통령
Answer:
도널드 트럼프 대통령은 현재 중국과의 무역 협상에서 "공정한 거래"를 목표로 하고 있으며, 중국 제품에 대한 높은 관세(현재 약 145%)를 "상당히 낮출 것"이라고 밝혔습니다. 그는 또한 우크라이나 전쟁과 관련하여 우크라이나 대통령 볼로디미르 젤렌스키를 비판하며, 그의 발언이 평화 협상에 해롭다고 언급했습니다. 트럼프의 지지율은 40%로 하락했으며, 이는 그의 두 번째 임기 100일을 앞두고 나타난 변화입니다.

**Source**
- [CNN - The latest on Trump’s presidency](https://www.cnn.com/politics/live-news/trump-presidency-news-04-23-25/index.html) 
- [CNN - Trump slams Zelensky for refusing to recognize Russian control of Crimea](https://edition.cnn.com/2025/04/23/europe/rubio-russia-ukraine-ceasefire-talks-intl-hnk/index.html) 
- [Politico - Trump to host dinner with top holders of his meme coin](https://www.politico.com/news/2025/04/23/trump-crypto-coin-dinner-00306707)
